# Email classificator Agent - v1

This notebook builds a basic ReAct agent architecture with memory, that classifies email passed in the prompt.

## Env Set up

In [0]:
!python3 --version

In [0]:
%pip install -U -qqqq langgraph langgraph-prebuilt langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core langchain-openai notebook langchain-tavily wikipedia trustcall langgraph-cli[inmem] transformers

%pip install -U -qqqq databricks-agents mlflow-skinny[databricks] databricks-langchain
dbutils.library.restartPython()

## Helper function for examples set up

In [0]:
from pyspark.sql.functions import *
def format_email_classification_request(email_sender:str, email_subject:str, email_body:str) -> str:

  """ The output of this function is a string which can be used to send a classification request to the Agent """
  
  request_template = "We received from: `{sender}` the following email. Here's the email subject: `{subject}` and here's the email body: `{body}`"

  return request_template.format(sender = email_sender, subject = email_subject, body = email_body)

In [0]:
## Creating a dataframe with examples
columns = ["ID", "sender", "subject", "body"]
examples_ = [
  (1, "mario.rossi@gmail.com", "Address update", "Dear sir or madam, I would like to get my residence address updated to: v. bella 1, Milano (MI), Italia"),

  (2, "alberto.rossi@gmail.com", "Shipment status", "Hello, I'd like to get an updated delivery date for shipment ID-45892171, given that I didn't receive it yet. Thank you"),

  (3, "alberto.rossi@gmail.com", "RE:Shipment status", "Understood, I will wait a couple of days thanks")
]
df_example = spark.createDataFrame(examples_, columns)
display(df_example)

In [0]:
example_row = df_example.filter(col("ID") == 2).first()
print(format_email_classification_request(example_row.sender, example_row.subject, example_row.body))

## Classifier Agent

In [0]:
## Set up the output schema for the classification
from pydantic import BaseModel, Field
from typing import Literal

class LabelOutput(BaseModel):
  label: Literal["Others", "Spam", "Claim", "General Enquiry", "Profile Update", "Communication Unsubscription"] = Field(
    description="Label assigned to the email",
    default="Others")
  retional: str = Field(description="Reasoning behind the label choice")